In [ ]:
from dataclasses import dataclass
from pathlib import Path

In [ ]:
import os

In [ ]:
%pwd

In [ ]:
os.chdir("../")

In [ ]:
@dataclass(frozen=True)
class ModelConfiguration:
    root_dir:Path
    tokenizer_path:Path
    model_path:Path

In [ ]:
from src.constants import *
from src.utils.common import read_yaml,create_directories

In [ ]:
class ConfriguationManager:
    def __init__(self,config_file_path=CONFIG_FILE_PATH):
        self.config=read_yaml(config_file_path)


    def get_model_evaluation_config(self)->ModelConfiguration:
        config=self.config.ModelConfig

        model_config=ModelConfiguration(
            root_dir=config.root_dir,
            tokenizer_path=config.tokenizer_path,
            model_path=config.model_path
        )

        return model_config

In [ ]:
%pip install datasets

In [ ]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
from datasets import load_dataset,load_from_disk
import pandas as pd
from tqdm import tqdm
import torch

In [ ]:
device="cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
path="D:/fake_news_detector/dataset/data_ingestion/Combined.csv"
df=pd.read_csv(path)



In [ ]:
df.head(5)

In [ ]:
%pip install scikit-learn

In [ ]:
import torch.nn.functional as F

In [ ]:
class Model:
    def __init__(self):
        self.config = ConfriguationManager()
        model_configs = self.config.get_model_evaluation_config()

        self.tokenizer = AutoTokenizer.from_pretrained(model_configs.tokenizer_path)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_configs.model_path).to(device)

        self.model.eval()


    def get_output(self,text):
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256)
        
        with torch.no_grad():
            outputs = self.model(**inputs)
            logits = outputs.logits
            probs = F.softmax(logits, dim=1)
            predicted_class_id = torch.argmax(probs, dim=1).item()
            confidence = probs[0][predicted_class_id].item()


        labels = ["Fake","true"]
        predicted_label = labels[predicted_class_id]

        print(f"Prediction: {predicted_label} ({confidence*100:.2f}% confidence)")


In [ ]:
model=Model()

In [ ]:
text_true="""ASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a “fiscal conservative” on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBS’ “Face the Nation,” drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a fight likely to be linked to other issues, such as immigration policy, even as the November congressional election campaigns approach in which Republicans will seek to keep control of Congress. President Donald Trump and his Republicans want a big budget increase in military spending, while Democrats also want proportional increases for non-defense “discretionary” spending on programs that support education, scientific research, infrastructure, public health and environmental protection. “The (Trump) administration has already been willing to say: ‘We’re going to increase non-defense discretionary spending ... by about 7 percent,’” Meadows, chairman of the small but influential House Freedom Caucus, said on the program. “Now, Democrats are saying that’s not enough, we need to give the government a pay raise of 10 to 11 percent. For a fiscal conservative, I don’t see where the rationale is. ... Eventually you run out of other people’s money,” he said. Meadows was among Republicans who voted in late December for their party’s debt-financed tax overhaul, which is expected to balloon the federal budget deficit and add about $1.5 trillion over 10 years to the $20 trillion national debt. “It’s interesting to hear Mark talk about fiscal responsibility,” Democratic U.S. Representative Joseph Crowley said on CBS. Crowley said the Republican tax bill would require the  United States to borrow $1.5 trillion, to be paid off by future generations, to finance tax cuts for corporations and the rich. “This is one of the least ... fiscally responsible bills we’ve ever seen passed in the history of the House of Representatives. I think we’re going to be paying for this for many, many years to come,” Crowley said. Republicans insist the tax package, the biggest U.S. tax overhaul in more than 30 years,  will boost the economy and job growth. House Speaker Paul Ryan, who also supported the tax bill, recently went further than Meadows, making clear in a radio interview that welfare or “entitlement reform,” as the party often calls it, would be a top Republican priority in 2018. In Republican parlance, “entitlement” programs mean food stamps, housing assistance, Medicare and Medicaid health insurance for the elderly, poor and disabled, as well as other programs created by Washington to assist the needy. Democrats seized on Ryan’s early December remarks, saying they showed Republicans would try to pay for their tax overhaul by seeking spending cuts for social programs. But the goals of House Republicans may have to take a back seat to the Senate, where the votes of some Democrats will be needed to approve a budget and prevent a government shutdown. Democrats will use their leverage in the Senate, which Republicans narrowly control, to defend both discretionary non-defense programs and social spending, while tackling the issue of the “Dreamers,” people brought illegally to the country as children. Trump in September put a March 2018 expiration date on the Deferred Action for Childhood Arrivals, or DACA, program, which protects the young immigrants from deportation and provides them with work permits. The president has said in recent Twitter messages he wants funding for his proposed Mexican border wall and other immigration law changes in exchange for agreeing to help the Dreamers. Representative Debbie Dingell told CBS she did not favor linking that issue to other policy objectives, such as wall funding. “We need to do DACA clean,” she said.  On Wednesday, Trump aides will meet with congressional leaders to discuss those issues. That will be followed by a weekend of strategy sessions for Trump and Republican leaders on Jan. 6 and 7, the White House said. Trump was also scheduled to meet on Sunday with Florida Republican Governor Rick Scott, who wants more emergency aid. The House has passed an $81 billion aid package after hurricanes in Florida, Texas and Puerto Rico, and wildfires in California. The package far exceeded the $44 billion requested by the Trump administration. The Senate has not yet voted on the aid"""

In [ ]:
model.get_output(text_true)